In [1]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow==2.7.0
!pip install stanza
!pip install transformers
!pip install tensorflow-addons
!pip install nltk
!pip install textacy

     |████████████████████████████████| 3.8 MB 14.4 MB/s 
     |████████████████████████████████| 67 kB 6.3 MB/s 
     |████████████████████████████████| 6.5 MB 58.0 MB/s 
     |████████████████████████████████| 596 kB 64.6 MB/s 
     |████████████████████████████████| 895 kB 59.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 14.4 MB/s 
     |████████████████████████████████| 489.6 MB 19 kB/s 
     |████████████████████████████████| 463 kB 77.3 MB/s 
     |████████████████████████████████| 1.3 MB 47.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully

In [3]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
from textacy.datasets.supreme_court import SupremeCourt
import numpy as np
import re
import unicodedata
import nltk
#from transformers import pipeline
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input, BatchNormalization
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import numpy as np
import gc
import math
import json
import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from transformers import TFRobertaModel,RobertaTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import RandomUniform

from numpy.random import seed
import random as python_random
import os
import sys

np.random.seed(1)
python_random.seed(1)
tf.random.set_seed(1)

In [10]:
!cp "/content/drive/My Drive/labels_sc.txt" "./labels_sc.txt"
!cp "/content/drive/My Drive/labels_sc_279.txt" "./labels_sc_279.txt"

In [24]:
sc = SupremeCourt()
print(sc.info)
sc.download()

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
print(issue_codes)
issue_codes.sort()
issue_codes = [str(ic) for ic in issue_codes]

labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
print(labels_index)
count=0

for record in sc.records():
        
        #print(record)
        count=count+1
        #print(count)
        if record[1]['issue'] == None: # some cases have None as an issue
            labels.append(labels_index['-1'])
        else:
            labels.append(labels_index[record[1]['issue'][:-4]])
        
        temp=record[0].split("Footnotes")[0]
        sentence=[]
        for word in temp.split():
            if any(ch.isdigit() for ch in word):
                continue
            else:
                sentence.append(word)
        new_sen=' '.join(sentence)
        new_new_sen=new_sen.split()
        new_new_sen=new_new_sen[0:512]
        new_new_sen=' '.join(new_new_sen)
        
        texts.append(new_new_sen)
   
len_list = [len(ele.split()) for ele in texts]

print(labels)
print(len(labels))

res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
print("Average Length %s" % res) 
print('Found %s texts.' % len(texts))
print('Found %s labels.' % len(labels_index))

temp_file = open("labels_sc.txt", "r")

data = temp_file.read()
label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]

    

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
[8, 1, 8, 2, 8, 8, 8, 9, 7, 8, 1, 2, 1, 1, 8, 4, 8, 8, 12, 3, 3, 7, 3, 12, 1, 8, 8, 8, 8, 8, 8, 8, 1, 9, 5, 9, 9, 9, 11, 8, 8, 8, 4, 8, 8, 8, 8, 1, 3, 9, 3, 7, 1, 2, 9, 9, 7, 8, 8, 1, 10, 7, 8, 8, 9, 8, 7, 9, 9, 12, 7, 2, 8, 1, 11, 11, 1, 7, 7, 12, 1, 9, 8, 10, 12, 7, 8, 2, 8, 9, 9, 1, 8, 9, 1, 7, 12, 10, 10, 10, 8, 3, 7, 9, 8, 9, 1, 8, 8, 2, 7, 2, 9, 9, 11, 8, 8, 12, 12, 7, 8, 12, 4, 9, 3, 1, 12, 1, 1, 8, 8, 3, 8, 8, 8, 8, 9, 1, 8, 8, 10, 1, 8, 2, 8, 8, 7, 3, 8, 2, 4, 4, 9, 8, 10, 12, 12, 1, 1, 9, 1, 1, 1, 8, 2, 2, 8, 1, 1, 2, 2, 1, 2, 8, 1, 9, 9, 8, 8, 4, 2, 9, 9, 8, 3, 4, 3, 1, 8, 8, 2, 1, 9, 7, 8

In [25]:
summarized_data = pd.DataFrame(texts,
               columns =['text'])
summarized_data['label'] = label_list
print(summarized_data)

                                                   text  label
0     [ Halliburton Oil Well Cementing Co. v. Walker...      8
1     Rehearing Denied Dec. See . Mr.Claude T. Barne...      1
2     Rehearing Denied Dec. See . Appeal from the Di...      8
3     Mr. Walter J. Cummings, Jr., of Washington, D....      2
4     Mr.A. Devitt Vaneck, of Washington, D.C., for ...      8
...                                                 ...    ...
8414  Opinion reported: Ante, p. DECREE It is ordere...     11
8415  In this dispute between Utah and the United St...     10
8416  The United States, to the exclusion of defenda...     10
8417  Louisiana's exception to the portion of the Sp...     11
8418  To resolve a dispute over the ownership of cer...      9

[8419 rows x 2 columns]


In [26]:
gc.collect()
tf.keras.backend.clear_session()

In [27]:
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [28]:
max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
len(sentences),len(labels)

(8419, 8419)

In [29]:
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    #dbert_layer = dbert_model(inps, attention_mask=masks,return_dict=True)
    #lstm = layers.LSTM(128,return_sequences=False)(dbert_layer.last_hidden_state)
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    #dense_0 = Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001))(lstm)
    dropout_0= Dropout(0.5)(dense_0)
    #dropout_0= Dropout(0.2)(dense_0)
    #batch=BatchNormalization()(dense_0)
    #dense_1 = Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #drop_1=layers.Dropout(0.4)(dense_1)
    pred = Dense(15, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    #pred = Dense(279, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

In [30]:
model_0=create_model()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 512, 768),                                                   
                                 hidden_states=None                                           

In [31]:
input_ids=[]
attention_masks=[]

for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])
input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [32]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)

In [33]:
log_dir='dbert_model'
model_save_path='./dbert_model_5.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
#optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5,clipnorm=1.)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
model_0.compile(loss=loss,optimizer=optimizer, metrics=[metric])


In [34]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 11 00:20:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    39W / 300W |   8851MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [35]:
history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=4,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/4


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


412/948 [============>.................] - ETA: 1:27 - loss: 7.5257 - accuracy: 0.5382

KeyboardInterrupt: ignored